In [1]:
import os
import xml.etree.ElementTree
# import cv2
import numpy
from utils import config
import posixpath

In [2]:
def load_image(file_name):
    path = posixpath.join(config.data_dir, config.image_dir, file_name + '.jpg')
    image = cv2.imread(path)
    return image


def load_label(file_name, split='train'):
    # Construct the new path for the label file
    path = posixpath.join(config.data_dir, config.label_dir, split, file_name + '.txt')
    
    boxes = []
    
    # Read the text file line by line
    with open(path, 'r') as f:
        for line in f:
            # Split the line into coordinates
            _, x_min, y_min, x_max, y_max = line.strip().split()
            x_min = float(x_min)
            y_min = float(y_min)
            x_max = float(x_max)
            y_max = float(y_max)

            boxes.append([x_min, y_min, x_max, y_max])
   
    boxes = numpy.asarray(boxes, numpy.float32)    
    return boxes

In [3]:
path = "Dataset/labels/valid/vid_4_700.txt"
with open(path, 'r') as f:
    for line in f:
        # print(line)
        _, x_min, y_min, x_max, y_max = line.strip().split()
        print(x_min, y_min, x_max, y_max)

0.562590448668639 0.6324806949999999 0.38422575976331363 0.2265122263157895
0.9337916063609467 0.564913127368421 0.13241678727810652 0.17631917631578944


In [4]:
# file_names = []
# with open(os.path.join(config.data_dir, 'train.txt')) as f:
#     for file_name in f.readlines():
#         image_path = os.path.join(config.data_dir, config.image_dir, file_name.rstrip() + '.jpg')
#         label_path = os.path.join(config.data_dir, config.label_dir, file_name.rstrip() + '.xml')
#         if os.path.exists(image_path) and os.path.exists(label_path):
#             if os.path.exists(os.path.join(config.data_dir, 'TF')):
#                 file_names.append(os.path.join(config.data_dir, 'TF', file_name.rstrip() + '.tf'))
#             else:
#                 file_names.append(file_name.rstrip())
                
# print(file_names)

In [5]:

image_path = posixpath.join(config.data_dir, config.image_dir, 'train')
label_path = posixpath.join(config.data_dir, config.label_dir, 'train')

image_files = [os.path.splitext(file_name)[0] for file_name in os.listdir(image_path) if file_name.lower().endswith('.jpg')]
label_files = [os.path.splitext(file_name)[0] for file_name in os.listdir(label_path) if file_name.lower().endswith('.txt')]

file_names = list(set(image_files) & set(label_files))
len(file_names), len(image_files), len(label_files)

(284, 338, 284)

In [6]:
from utils import config, util
from tensorflow.keras import utils
import tensorflow as tf


class Generator(utils.Sequence):
    def __init__(self, file_names):
        self.file_names = file_names

    def __len__(self):
        return int(numpy.floor(len(self.file_names) / config.batch_size))

    def __getitem__(self, index):
        image = util.load_image(self.file_names[index])
        boxes = util.load_label(self.file_names[index])
        image, boxes = util.resize(image, boxes)
        # image, boxes = util.random_flip(image, boxes)

        image = image[:, :, ::-1].astype(numpy.float32)
        image = image / 255.0
        y_true_1, y_true_2, y_true_3 = util.process_box(boxes)
        return image, y_true_1, y_true_2, y_true_3

    def on_epoch_end(self):
        numpy.random.shuffle(self.file_names)


def input_fn(file_names):
    # def generator_fn():
        generator = utils.OrderedEnqueuer(Generator(file_names), True)
        # generator.start(workers=min(os.cpu_count(), config.batch_size))
        while True:
            image, y_true_1, y_true_2, y_true_3 = generator.get().__next__()
            print(image, y_true_1, y_true_2, y_true_3)
            yield image, y_true_1, y_true_2, y_true_3
            
            
input_fn(file_names)

<generator object input_fn at 0x000002653F301EE0>

In [7]:
index = 0
print(file_names[index])
image = util.load_image(file_names[index])
boxes = util.load_label(file_names[index])
image, boxes = util.resize(image, boxes)
# image, boxes = util.random_flip(image, boxes)

image = image[:, :, ::-1].astype(numpy.float32)
image = image / 255.0
y_true_1, y_true_2, y_true_3 = util.process_box(boxes)
print(image, y_true_1, y_true_2, y_true_3)


vid_4_6380
[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]] [[[[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  ...

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0

In [8]:
# # Create an iterator for the dataset
# iterator = iter(dataset)

# # Get one sample from the dataset
# sample = next(iterator)

# # Unpack the sample
# image, y_true_1, y_true_2, y_true_3 = sample